# joshpy Registry Demo: Parameter Sweep Analysis

This notebook demonstrates the full workflow for running parameter sweeps with Josh simulations,
tracking results in a DuckDB registry, and analyzing the outcomes.

We'll sweep over 10 values of `maxGrowth` (from 10 to 100 meters) and show that:
- Higher growth rates lead to higher average tree heights
- Higher growth rates lead to more variance in tree heights

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

# joshpy imports
from joshpy.jobs import (
    JobConfig,
    SweepConfig,
    SweepParameter,
    JobExpander,
    JobRunner,
)
from joshpy.registry import RunRegistry, RegistryCallback
from joshpy.jar import JarMode

## 1. Setup Paths and Configuration

In [ ]:
# Paths to simulation files
examples_dir = Path("examples")
source_path = examples_dir / "hello_cli_configurable.josh"
template_path = examples_dir / "templates" / "sweep_config.jshc.j2"

# Verify files exist
assert source_path.exists(), f"Source not found: {source_path}"
assert template_path.exists(), f"Template not found: {template_path}"

print(f"Source file: {source_path}")
print(f"Template file: {template_path}")

In [ ]:
# Display the template
print("Template contents:")
print(template_path.read_text())

## 2. Define the Parameter Sweep

We'll sweep over 10 values of `maxGrowth` from 10 to 100 meters.
Each tree grows by a random amount between 0 and `maxGrowth` per timestep.

In [ ]:
# Define parameter sweep: 10 values from 10 to 100 meters
max_growth_values = list(range(10, 101, 10))  # [10, 20, 30, ..., 100]
print(f"maxGrowth values to test: {max_growth_values}")

# Create job configuration
config = JobConfig(
    template_path=template_path,
    source_path=source_path,
    simulation="Main",
    replicates=3,  # Run 3 replicates per configuration for variance
    sweep=SweepConfig(
        parameters=[
            SweepParameter(name="maxGrowth", values=max_growth_values),
        ]
    ),
)

print(f"\nJob configuration:")
print(config.to_yaml())

## 3. Initialize the Registry

Create a DuckDB registry to track our experiment.

In [ ]:
# Create registry (use in-memory for this demo, or a file path for persistence)
registry = RunRegistry(":memory:")  # Use "growth_experiment.duckdb" for persistence

# Create a session for this experiment
session_id = registry.create_session(
    experiment_name="growth_rate_sensitivity",
    simulation="Main",
    total_jobs=len(max_growth_values),
    total_replicates=len(max_growth_values) * 3,
    template_path=str(template_path),
    metadata={
        "description": "Testing effect of maxGrowth on tree height",
        "parameter_range": "10-100 meters",
    }
)

print(f"Created session: {session_id}")

## 4. Expand Jobs and Register Configs

In [ ]:
# Expand the configuration into concrete jobs
expander = JobExpander()
job_set = expander.expand(config)

print(f"Expanded to {len(job_set)} jobs")

# Register each config in the registry
for job in job_set.jobs:
    registry.register_config(
        session_id=session_id,
        config_hash=job.config_hash,
        config_content=job.config_content,
        parameters=job.parameters,
    )
    print(f"  Registered config: maxGrowth={job.parameters['maxGrowth']} (hash: {job.config_hash})")

## 5. Run the Sweep with Registry Tracking

In [ ]:
# Update session status to running
registry.update_session_status(session_id, "running")

# Create runner using local jar
runner = JobRunner(josh_jar=JarMode.LOCAL)

# Create callback to track runs in registry
callback = RegistryCallback(registry, session_id)

# Track progress
def on_complete_with_logging(result):
    status = "OK" if result.success else "FAIL"
    params = result.job.parameters
    print(f"  [{status}] maxGrowth={params.get('maxGrowth')}")
    if not result.success:
        print(f"    Error: {result.stderr[:200]}")
    # Also record in registry
    callback(result)

print("Running parameter sweep...")
print("=" * 40)
results = runner.run_all(job_set, on_complete=on_complete_with_logging, cleanup=False)
print("=" * 40)

# Update session status
all_success = all(r.success for r in results)
registry.update_session_status(session_id, "completed" if all_success else "failed")

print(f"\nCompleted: {sum(1 for r in results if r.success)}/{len(results)} succeeded")

## 6. Query the Registry

In [ ]:
# Get session summary
summary = registry.get_session_summary(session_id)

print("Session Summary:")
print(f"  Experiment: {summary.experiment_name}")
print(f"  Status: {summary.status}")
print(f"  Total jobs: {summary.total_jobs}")
print(f"  Runs completed: {summary.runs_completed}")
print(f"  Runs succeeded: {summary.runs_succeeded}")
print(f"  Runs failed: {summary.runs_failed}")

In [ ]:
# Query runs by parameter - e.g., get all runs with maxGrowth=50
runs_50 = registry.get_runs_by_parameters(maxGrowth=50)
print(f"Runs with maxGrowth=50: {len(runs_50)}")
for run in runs_50:
    print(f"  Run {run['run_id'][:8]}... exit_code={run['exit_code']}")

In [ ]:
# Export all results as DataFrame
results_df = registry.export_results_df(session_id)
print(f"Results DataFrame shape: {results_df.shape}")
results_df.head()

## 7. Parse Simulation Output Files

The simulation exports average height data to CSV files. Let's parse them.

In [ ]:
# The simulation writes output to /tmp/hello_josh.csv
# For each job, we need to read this file and associate it with the maxGrowth value

output_file = Path("/tmp/hello_josh.csv")

# Collect results from each successful run
all_data = []

for result in results:
    if not result.success:
        continue
    
    max_growth = result.job.parameters["maxGrowth"]
    
    # Read the output file (note: each run overwrites it, so we read right after running)
    # For this demo, we'll parse from stdout which contains the export path info
    # In a real scenario, you'd save outputs to unique paths using path templates
    
    if output_file.exists():
        try:
            df = pd.read_csv(output_file)
            df["maxGrowth"] = max_growth
            df["config_hash"] = result.job.config_hash
            all_data.append(df)
        except Exception as e:
            print(f"Error reading output for maxGrowth={max_growth}: {e}")

if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"Combined output data: {len(combined_df)} rows")
    combined_df.head()

In [ ]:
# Since the simulation overwrites the output file each time, let's re-run
# and capture the data properly by reading after each job

# Re-expand jobs
job_set2 = expander.expand(config)

all_simulation_data = []

print("Re-running to capture output data...")
for job in job_set2.jobs:
    result = runner.run(job)
    
    if result.success and output_file.exists():
        try:
            df = pd.read_csv(output_file)
            df["maxGrowth"] = job.parameters["maxGrowth"]
            df["config_hash"] = job.config_hash
            all_simulation_data.append(df.copy())
            print(f"  maxGrowth={job.parameters['maxGrowth']}: captured {len(df)} rows")
        except Exception as e:
            print(f"  maxGrowth={job.parameters['maxGrowth']}: error - {e}")
    else:
        status = "success" if result.success else "failed"
        print(f"  maxGrowth={job.parameters['maxGrowth']}: {status}, no output")

job_set2.cleanup()

if all_simulation_data:
    simulation_df = pd.concat(all_simulation_data, ignore_index=True)
    print(f"\nTotal simulation data: {len(simulation_df)} rows")
else:
    print("No simulation data captured")
    simulation_df = pd.DataFrame()

In [ ]:
# Display the simulation data
if not simulation_df.empty:
    print("Simulation output columns:", simulation_df.columns.tolist())
    simulation_df.head(10)

## 8. Analyze Results: Effect of maxGrowth on Tree Height

Let's analyze the relationship between `maxGrowth` and tree height.

In [ ]:
if not simulation_df.empty:
    # Get the final step data for each maxGrowth value
    # The simulation runs for 10 steps (0-10)
    final_step = simulation_df.groupby('maxGrowth').last().reset_index()
    
    # Also compute statistics across all steps
    height_col = [c for c in simulation_df.columns if 'height' in c.lower()]
    if height_col:
        height_col = height_col[0]
        print(f"Using height column: {height_col}")
        
        stats = simulation_df.groupby('maxGrowth')[height_col].agg(['mean', 'std', 'min', 'max'])
        stats.columns = ['Mean Height', 'Std Dev', 'Min', 'Max']
        print("\nHeight Statistics by maxGrowth:")
        print(stats.round(2))
    else:
        print("Height column not found. Available columns:", simulation_df.columns.tolist())

In [ ]:
if not simulation_df.empty and height_col:
    # Get final timestep heights for analysis
    final_heights = simulation_df[simulation_df['step'] == simulation_df['step'].max()].copy()
    
    # Calculate mean and variance per maxGrowth
    analysis = final_heights.groupby('maxGrowth')[height_col].agg(['mean', 'var', 'std']).reset_index()
    analysis.columns = ['maxGrowth', 'mean_height', 'variance', 'std_dev']
    
    print("\nFinal Step Height Analysis:")
    print("=" * 60)
    print(analysis.to_string(index=False))
    
    # Calculate correlation
    corr_mean = analysis['maxGrowth'].corr(analysis['mean_height'])
    corr_var = analysis['maxGrowth'].corr(analysis['variance']) if analysis['variance'].notna().any() else 0
    
    print(f"\nCorrelation Analysis:")
    print(f"  maxGrowth vs Mean Height: r = {corr_mean:.4f}")
    print(f"  maxGrowth vs Variance:    r = {corr_var:.4f}")

## 9. Visualize the Results

In [ ]:
try:
    import matplotlib.pyplot as plt
    HAS_MATPLOTLIB = True
except ImportError:
    HAS_MATPLOTLIB = False
    print("matplotlib not installed - skipping plots")

In [ ]:
if HAS_MATPLOTLIB and not simulation_df.empty and 'analysis' in dir():
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot 1: Mean height vs maxGrowth
    ax1 = axes[0]
    ax1.plot(analysis['maxGrowth'], analysis['mean_height'], 'bo-', linewidth=2, markersize=8)
    ax1.set_xlabel('maxGrowth (meters)', fontsize=12)
    ax1.set_ylabel('Mean Final Height (meters)', fontsize=12)
    ax1.set_title('Higher Growth Rate → Taller Trees', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(analysis['maxGrowth'], analysis['mean_height'], 1)
    p = np.poly1d(z)
    ax1.plot(analysis['maxGrowth'], p(analysis['maxGrowth']), 'r--', alpha=0.7, label='Linear fit')
    ax1.legend()
    
    # Plot 2: Variance vs maxGrowth  
    ax2 = axes[1]
    if analysis['std_dev'].notna().any():
        ax2.bar(analysis['maxGrowth'], analysis['std_dev'], width=7, color='orange', alpha=0.7)
        ax2.set_xlabel('maxGrowth (meters)', fontsize=12)
        ax2.set_ylabel('Standard Deviation (meters)', fontsize=12)
        ax2.set_title('Higher Growth Rate → More Variance', fontsize=14)
        ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig('growth_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\nPlot saved to growth_analysis.png")

In [ ]:
if HAS_MATPLOTLIB and not simulation_df.empty and height_col:
    # Plot height trajectories over time for different maxGrowth values
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(max_growth_values)))
    
    for i, mg in enumerate(max_growth_values):
        subset = simulation_df[simulation_df['maxGrowth'] == mg]
        if not subset.empty:
            ax.plot(subset['step'], subset[height_col], 'o-', 
                   color=colors[i], label=f'maxGrowth={mg}', alpha=0.8)
    
    ax.set_xlabel('Simulation Step', fontsize=12)
    ax.set_ylabel('Average Tree Height (meters)', fontsize=12)
    ax.set_title('Tree Height Over Time by Growth Rate', fontsize=14)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('height_trajectories.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\nPlot saved to height_trajectories.png")

## 10. Registry Queries with DuckDB SQL

You can also query the registry directly using SQL.

In [ ]:
# Direct SQL queries on the registry
print("Sessions in registry:")
sessions_df = registry.conn.execute("""
    SELECT session_id, experiment_name, simulation, status, total_jobs
    FROM sweep_sessions
""").df()
display(sessions_df)

In [ ]:
# Query configs and their parameters
print("\nConfigs with parameters:")
configs_df = registry.conn.execute("""
    SELECT 
        config_hash,
        json_extract_string(parameters, '$.maxGrowth') as maxGrowth,
        created_at
    FROM job_configs
    ORDER BY CAST(json_extract_string(parameters, '$.maxGrowth') AS INTEGER)
""").df()
display(configs_df)

In [ ]:
# Query runs with success/failure breakdown
print("\nRuns by exit code:")
runs_df = registry.conn.execute("""
    SELECT 
        c.config_hash,
        json_extract_string(c.parameters, '$.maxGrowth') as maxGrowth,
        r.exit_code,
        CASE WHEN r.exit_code = 0 THEN 'success' ELSE 'failed' END as status
    FROM job_runs r
    JOIN job_configs c ON r.config_hash = c.config_hash
    ORDER BY CAST(json_extract_string(c.parameters, '$.maxGrowth') AS INTEGER)
""").df()
display(runs_df)

## 11. Summary

This demo showed:

1. **Parameter Sweep Setup**: Using `JobConfig` with `SweepConfig` and `SweepParameter`
2. **Registry Tracking**: Creating sessions, registering configs, and tracking runs
3. **Execution with Callbacks**: Using `RegistryCallback` with `JobRunner`
4. **Query Methods**: `get_session_summary()`, `get_runs_by_parameters()`, `export_results_df()`
5. **Direct SQL Queries**: Using DuckDB SQL for custom analysis

**Key Finding**: As expected, higher `maxGrowth` values lead to:
- Higher average tree heights (linear relationship)
- Greater variance in tree heights (due to wider uniform distribution range)

In [ ]:
# Cleanup
registry.close()
print("Registry closed.")